# Baza BASiW
Przypadki i zgony w podziale na:
* Datę
* Województwo
* Powiat
* Wiek
* Płeć
* Stan zaszczepienia
* Choroby współistniejące
* Upośledzenie odporności
* Więcej...

## ŚCIĄGNIJ AKTUALNE DANE RĘCZNIE!!!
https://basiw.mz.gov.pl/index.html#/visualization?id=3761

In [1]:
"""
Set PYTHONPATH for modules in parallel directory

https://stackoverflow.com/questions/3108285/in-python-script-how-do-i-set-pythonpath/3108301

"""

import sys

try:
    sys.path.index('..') # Or os.getcwd() for this directory
except ValueError:
    sys.path.append('..') # Or os.getcwd() for this directory

#--------------------------------------------------------------
import pandas as pd
from share.helper_functions import was_modified_today, display_all
import plotly.express as px
import plotly.graph_objects as go
import os
import datetime


In [2]:
class RawDf:
    df : pd.DataFrame
    def __init__(self, df : pd.DataFrame) -> None:
        self.df=df

class BASiWformat:
    image_dir : str
    data_dir : str
    data_file_deaths : str
    data_file_cases : str
    teryt_file : str
    path_deaths : str
    path_cases : str
    path_teryt : str
    _dfd_raw : RawDf
    _dfc_raw : RawDf
    _df_teryt_raw : RawDf
    df_woj : pd.DataFrame
    teryt_dict : dict
    dfd : pd.DataFrame

    def __init__(self,
        image_dir : str,
        data_dir : str,
        data_file_deaths : str,
        data_file_cases : str,
        teryt_file : str
        ) -> None:

        self.image_dir=image_dir
        self.data_dir=data_dir
        self.data_file_deaths=data_file_deaths
        self.data_file_cases=data_file_cases
        self.teryt_file=teryt_file
        self.path_deaths = os.sep.join([self.data_dir,self.data_file_deaths])
        self.path_cases = os.sep.join([self.data_dir,self.data_file_cases])
        self.path_teryt = os.sep.join([self.data_dir,self.teryt_file])
        self.read_data()
        self.make_teryt_dict()
        self.dfd = self.format_df('data_rap_zgonu', self._dfd_raw)
        return


    def read_data(self) -> None:
        # if was_modified_today(path_deaths):
        #     print('Data up to date')
        #     dfd = pd.read_csv(path_deaths, encoding = 'cp1250', sep = ';')
        #     dfc = pd.read_csv(path_cases, encoding = 'cp1250', sep = ';')
        # else:
        #     print('Old data, need to download new data!')
        self._dfd_raw = RawDf(pd.read_csv(self.path_deaths, encoding = 'cp1250', sep = ';',low_memory=False))
        self._dfc_raw= RawDf(pd.read_csv(self.path_cases, encoding = 'cp1250', sep = ';', low_memory=False))
        self._df_teryt_raw= RawDf(pd.read_csv(self.path_teryt, sep = ';'))
        return
        
    def make_teryt_dict(self) -> None:
        self.df_woj = self._df_teryt_raw.df.query('NAZWA_DOD == "województwo"')[['WOJ', 'NAZWA']]
        self.df_woj['NAZWA'] = self.df_woj['NAZWA'].str.title()
        self.df_woj.index = self.df_woj.pop('WOJ')
        self.teryt_dict = self.df_woj.to_dict()['NAZWA']
        return

    def format_df(self, col : str, df_raw : RawDf) -> pd.DataFrame:
        df = df_raw.df.copy(deep=True)
        df[col] = pd.to_datetime(df[col], format = "%Y-%m-%d")
        df['Województwo'] = df['teryt_woj']
        df['Województwo'].replace(self.teryt_dict, inplace=True)
        return df

    def dfd_from(self, start_date : datetime.date) -> pd.DataFrame:
        return self.dfd[self.dfd['data_rap_zgonu'] >= pd.to_datetime(start_date)]
        

In [3]:
basiw = BASiWformat(
    image_dir = './images',
    data_dir = './basiw_data',
    data_file_deaths = 'zgony.csv',
    data_file_cases = 'zakazenia.csv',
    teryt_file = 'TERC_Urzedowy_2021-12-14.csv'
    )

basiw.dfd


,data_rap_zgonu,teryt_woj,teryt_pow,plec,wiek,kat_wiek,czy_wspolistniejace,producent,dawka_ost,obniz_odpornosc,liczba_zaraportowanych_zgonow,w_pelni_zaszczepiony,Województwo
0,2021-01-02,4,407,K,67.0,65-74,T,NaN,NaN,N,1,N,Kujawsko-Pomorskie
1,2021-01-02,4,407,K,90.0,85-94,T,NaN,NaN,N,1,N,Kujawsko-Pomorskie
2,2021-01-02,4,407,M,69.0,65-74,T,NaN,NaN,N,1,N,Kujawsko-Pomorskie
3,2021-01-02,4,415,M,72.0,65-74,T,NaN,NaN,N,1,N,Kujawsko-Pomorskie
4,2021-01-02,4,415,M,77.0,75-84,T,NaN,NaN,N,1,N,Kujawsko-Pomorskie
...,...,...,...,...,...,...,...,...,...,...,...,...,...
58975,2021-12-13,32,3261,M,83.0,75-84,T,NaN,NaN,N,1,N,Zachodniopomorskie
58976,2021-12-13,32,3261,M,85.0,85-94,T,Pfizer,pelna_dawka,N,1,T,Zachodniopomorskie
58977,2021-12-13,32,3262,M,73.0,65-74,T,Pfizer,pelna_dawka,N,1,T,Zachodniopomorskie
58978,2021-12-13,32,3262,M,78.0,75-84,T,Pfizer,pelna_dawka,N,1,T,Zachodniopomorskie


In [4]:
wave4_date = datetime.date(2021, 7, 14)
dfd4 = basiw.dfd_from(wave4_date)
dfd4


,data_rap_zgonu,teryt_woj,teryt_pow,plec,wiek,kat_wiek,czy_wspolistniejace,producent,dawka_ost,obniz_odpornosc,liczba_zaraportowanych_zgonow,w_pelni_zaszczepiony,Województwo
45248,2021-07-14,2,201,K,95.0,95+,N,NaN,NaN,N,1,N,Dolnośląskie
45249,2021-07-14,4,410,K,89.0,85-94,T,NaN,NaN,N,1,N,Kujawsko-Pomorskie
45250,2021-07-14,14,1412,K,45.0,45-54,N,NaN,NaN,N,1,N,Mazowieckie
45251,2021-07-14,16,1610,M,65.0,65-74,T,Astra Zeneca,jedna_dawka,N,1,N,Opolskie
45252,2021-07-14,18,1808,M,74.0,65-74,T,NaN,NaN,N,1,N,Podkarpackie
...,...,...,...,...,...,...,...,...,...,...,...,...,...
58975,2021-12-13,32,3261,M,83.0,75-84,T,NaN,NaN,N,1,N,Zachodniopomorskie
58976,2021-12-13,32,3261,M,85.0,85-94,T,Pfizer,pelna_dawka,N,1,T,Zachodniopomorskie
58977,2021-12-13,32,3262,M,73.0,65-74,T,Pfizer,pelna_dawka,N,1,T,Zachodniopomorskie
58978,2021-12-13,32,3262,M,78.0,75-84,T,Pfizer,pelna_dawka,N,1,T,Zachodniopomorskie


### Liczba zgonów na COVID-19 w województwach na 100 tys. mieszkańców<br> w podziale na w pełni zaszczepionych i nie w pełni zaszczepionych<br> (Od 14.07.2021)

In [5]:
dfpopwoj = pd.read_csv('/home/ochab/koronawirus_PAN/gov.pl/bitbucket/govpl/GUS/gus_data/wojewodztwa_ludnosc_2021_GUS.csv')
dfpopwoj = pd.read_csv('/home/ochab/koronawirus_PAN/gov.pl/bitbucket/govpl/GUS/gus_data/wojewodztwa_ludnosc_2021_GUS.csv')
dfpopwoj['Województwo'] = dfpopwoj['Województwo'].str.title()
dfpopwoj.drop(['TERYT'], axis='columns', inplace=True)
dfpopwoj.index = dfpopwoj['Województwo']
dfpopwoj.drop(['Województwo'], axis='columns' , inplace = True)
pop_dict = dfpopwoj.to_dict()['Ludność']
# pop_dict

In [6]:
 def prepare_df(deaths_df: pd.DataFrame, from_date : datetime.date) -> pd.DataFrame:
    base_df = deaths_df[deaths_df['data_rap_zgonu'] >= pd.to_datetime(from_date)]

    df = base_df.groupby('Województwo').sum()
    df.drop(['wiek', 'teryt_pow', 'teryt_woj'], axis = 'columns' , inplace=True)
    df['Województwo'] = df.index
    df['Ludność'] = df['Województwo'].replace(pop_dict)
    # df

    Q = f'(w_pelni_zaszczepiony == "N")'

    df1 = base_df.query(Q).groupby('Województwo').sum()
    df1.drop(['wiek', 'teryt_pow', 'teryt_woj'], axis = 'columns' , inplace=True)
    df['Zgony nie w pełni zaszczepionych'] = df1

    Q = f'(w_pelni_zaszczepiony == "T")'

    df1 = base_df.query(Q).groupby('Województwo').sum()
    df1.drop(['wiek', 'teryt_pow', 'teryt_woj'], axis = 'columns' , inplace=True)
    df['Zgony w pełni zaszczepionych'] = df1




    df['Zgony nie w pełni zaszczepionych na 100 tys. mieszkańców'] = df['Zgony nie w pełni zaszczepionych'] / df['Ludność'] * 1e5
    df['Zgony w pełni zaszczepionych na 100 tys. mieszkańców'] = df['Zgony w pełni zaszczepionych'] / df['Ludność'] * 1e5

    df = df.sort_values(['Zgony nie w pełni zaszczepionych na 100 tys. mieszkańców'], ascending=False)
    return df

    
def plot_df(from_date : datetime.date, df : pd.DataFrame, image_dir : str) -> pd.DataFrame:
    nie = 'Zgony nie w pełni zaszczepionych na 100 tys. mieszkańców'
    tak = 'Zgony w pełni zaszczepionych na 100 tys. mieszkańców'

    fig = go.Figure(
        data=[
            go.Bar(
                name = nie, 
                x=df['Województwo'], 
                y=df[nie], 
                offsetgroup=1,
                text = df[nie]
                ),
            go.Bar(
                name = tak, 
                x=df['Województwo'], 
                y=df[tak], 
                offsetgroup=2,
                text = df[tak]
                )
        ]
    )

    fig.update_traces(
        texttemplate='%{text:2.1f}', 
        textposition='outside',
        )

    fig.update_layout(
        barmode='group', 
        width=800, 
        height=500,
        title = "Liczba zgonów na COVID-19 w województwach na 100 tys. mieszkańców<br>\
        w podziale na w pełni zaszczepionych i nie w pełni zaszczepionych<br>(Od " + str(from_date) + ")",
        yaxis_title="Liczba zgonów na 100 tys.",
        legend_title="",
        uniformtext_minsize=6, 
        uniformtext_mode='show',
        legend=dict(
            y=-0.5,
            xanchor="center",
            x=0.5
            ),
        title_xanchor='center',
        title_x = 0.5,
        title_font_size=16
        )
    fig.update_xaxes(tickangle=33)
    fig.show()
    fig.write_image(os.sep.join([image_dir, 'zgony_szczep_1e5_woj_od'+str(from_date)+'.jpg']), scale=4)
    return


In [7]:
from_date = datetime.date(2021,7,14)

df = prepare_df(basiw.dfd, from_date)
plot_df(from_date, df, basiw.image_dir)

### Liczba zgonów na COVID-19 w Polsce na 100 tys. mieszkańców<br> w podziale na grupy wiekowe<br> (Od 14.07.2021)

**Potrzebny jest słownik z danymi: Populacja Polski / regionu wg grup wiekowych !!!**
/home/ochab/koronawirus_PAN/gov.pl/bitbucket/govpl/Eurostat/eurostat_data/Poland_2020_population.csv

In [69]:
pop_file = "../Eurostat/eurostat_data/Poland_2020_population.csv"
pop_df = pd.read_csv(pop_file , index_col=0)
# print(pop_df['Population 2020 F'].sum() + pop_df['Population 2020 M'].sum())
# pop_df['Population 2020 T'].sum()

pop_df.rename(
    columns = {
        'Population 2020 T' : 'mieszkańców',
        'Population 2020 F' : 'kobiet',
        'Population 2020 M' : 'mężczyzn'
        }, inplace = True
    )
pop_df

,mieszkańców,kobiet,mężczyzn
Age group,,,
0-18,6909092.0,3362871.0,3546221.0
19-24,2391135.0,1169959.0,1221176.0
25-34,5378329.0,2637687.0,2740642.0
35-44,6101624.0,3008948.0,3092676.0
45-54,4789676.0,2395882.0,2393794.0
55-64,5104262.0,2668881.0,2435381.0
65-74,4195973.0,2362488.0,1833485.0
75-84,1912001.0,1218241.0,693760.0
85-94,756428.0,540453.0,215975.0


In [82]:
def group_by(base_df: pd.DataFrame, x_col: str, x_col_name : str, denominator_col : str) -> pd.DataFrame:
   df = base_df.groupby(x_col).sum()
   df.drop(['wiek', 'teryt_pow', 'teryt_woj'], axis = 'columns' , inplace=True)
   df = pd.concat([df,pop_df[denominator_col]], axis='columns')
   # display(df)

   Q = f'(w_pelni_zaszczepiony == "N")'

   df1 = base_df.query(Q).groupby(x_col).sum()
   df1.drop(['wiek', 'teryt_pow', 'teryt_woj'], axis = 'columns' , inplace=True)
   df['Zgony nie w pełni zaszczepionych '  + denominator_col] = df1
   
   
   Q = f'(w_pelni_zaszczepiony == "T")'

   df1 = base_df.query(Q).groupby(x_col).sum()
   df1.drop(['wiek', 'teryt_pow', 'teryt_woj'], axis = 'columns' , inplace=True)
   df['Zgony w pełni zaszczepionych '  + denominator_col] = df1
   df[x_col]=df.index
   df.rename({x_col : x_col_name}, axis=1, inplace=True)
   return df
 
 
def prepare_df2(deaths_df: pd.DataFrame, x_col: str, x_col_name : str, denominator_col : str, from_date : datetime.date) -> pd.DataFrame:
   base_df = deaths_df[deaths_df['data_rap_zgonu'] >= pd.to_datetime(from_date)]

   df = group_by(base_df, x_col, x_col_name, denominator_col)
   # display(df)

   df['Zgony nie w pełni zaszczepionych ' + denominator_col + ' na 100 tys. ' + denominator_col] =\
      df['Zgony nie w pełni zaszczepionych '  + denominator_col].to_numpy() / df[denominator_col].to_numpy() * 1e5
   # display(df)
   
   df['Zgony w pełni zaszczepionych ' + denominator_col + ' na 100 tys. ' + denominator_col] =\
      df['Zgony w pełni zaszczepionych '  + denominator_col].to_numpy() / df[denominator_col].to_numpy() * 1e5

   # df = df.sort_values(['Zgony nie w pełni zaszczepionych na 100 tys. mieszkańców'], ascending=False)
   return df
   

In [83]:
from_date = datetime.date(2021,7,14)

df = prepare_df2(
    deaths_df = basiw.dfd, 
    x_col ='kat_wiek', 
    x_col_name = 'Grupa wiekowa',  
    denominator_col='mieszkańców', 
    from_date=from_date
    )

df

,liczba_zaraportowanych_zgonow,mieszkańców,Zgony nie w pełni zaszczepionych mieszkańców,Zgony w pełni zaszczepionych mieszkańców,Grupa wiekowa,Zgony nie w pełni zaszczepionych mieszkańców na 100 tys. mieszkańców,Zgony w pełni zaszczepionych mieszkańców na 100 tys. mieszkańców
0-18,9,6909092.0,8,1.0,0-18,0.115789,0.014474
19-24,18,2391135.0,17,1.0,19-24,0.710959,0.041821
25-34,99,5378329.0,92,7.0,25-34,1.710568,0.130152
35-44,253,6101624.0,228,25.0,35-44,3.736710,0.409727
45-54,622,4789676.0,542,80.0,45-54,11.316006,1.670259
55-64,1465,5104262.0,1172,293.0,55-64,22.961204,5.740301
65-74,3565,4195973.0,2432,1133.0,65-74,57.960335,27.002080
75-84,3929,1912001.0,2714,1215.0,75-84,141.945532,63.545992
85-94,3536,756428.0,2622,914.0,85-94,346.629157,120.831064
95+,367,52344.0,299,68.0,95+,571.221152,129.909827


In [112]:
def plot_df2(x_col : str, denominator_col : str, title_str : str, filename_str : str,  from_date : datetime.date, df : pd.DataFrame, image_dir : str) -> pd.DataFrame:
    nie = 'Zgony nie w pełni zaszczepionych ' + denominator_col + ' na 100 tys. ' + denominator_col
    tak = 'Zgony w pełni zaszczepionych ' + denominator_col + ' na 100 tys. ' + denominator_col

    fig = go.Figure(
        data=[
            go.Bar(
                name = nie, 
                x=df[x_col], 
                y=df[nie], 
                offsetgroup=1,
                text = df[nie]
                ),
            go.Bar(
                name = tak, 
                x=df[x_col], 
                y=df[tak], 
                offsetgroup=2,
                text = df[tak]
                )
        ]
    )

    fig.update_traces(
        texttemplate='%{text:2.1f}', 
        textposition='outside',
        )

    fig.update_layout(
        barmode='group', 
        width=800, 
        height=500,
        title = "Liczba zgonów " + denominator_col + " na COVID-19 " + title_str + " na 100 tys. "+ denominator_col +
        "<br>w podziale na w pełni zaszczepionych i nie w pełni zaszczepionych<br>(od " + str(from_date) + ")",
        yaxis_title="Liczba zgonów na 100 tys.",
        legend_title="",
        uniformtext_minsize=6, 
        uniformtext_mode='show',
        legend=dict(
            y=-0.5,
            xanchor="center",
            x=0.5
            ),
        title_xanchor='center',
        title_x = 0.5,
        title_font_size=16
        )
    fig.update_xaxes(tickangle=33)
    fig.show()
    fig.write_image(os.sep.join([image_dir, 'zgony_szczep_1e5_'+ filename_str + '_od'+str(from_date)+'.jpg']), scale=4)
    return

In [115]:
x_col = "Grupa wiekowa"
title_str = "w grupach wiekowych w Polsce," #"w województwach"
filename_str = "wiek_ogolem"
denominator_col = "mieszkańców"
# title = "Liczba zgonów " + denominator_col + " na COVID-19 " + title_str + " na 100 tys. "+ denominator_col +\
        # "<br>w podziale na w pełni zaszczepionych i nie w pełni zaszczepionych<br>(od " + str(from_date) + ")",
df_to_plot = df[:-1]

plot_df2(
    x_col=x_col, 
    denominator_col=denominator_col, 
    title_str=title_str , 
    filename_str=filename_str,  
    from_date = from_date, 
    df = df_to_plot, 
    image_dir = basiw.image_dir
    )

### Liczba zgonów na COVID-19 w Polsce, według płci, na 100 tys. mieszkańców<br> w podziale na grupy wiekowe<br> (Od 14.07.2021) 

In [101]:
basiw.dfd.query('plec=="M"')

,data_rap_zgonu,teryt_woj,teryt_pow,plec,wiek,kat_wiek,czy_wspolistniejace,producent,dawka_ost,obniz_odpornosc,liczba_zaraportowanych_zgonow,w_pelni_zaszczepiony,Województwo
2,2021-01-02,4,407,M,69.0,65-74,T,NaN,NaN,N,1,N,Kujawsko-Pomorskie
3,2021-01-02,4,415,M,72.0,65-74,T,NaN,NaN,N,1,N,Kujawsko-Pomorskie
4,2021-01-02,4,415,M,77.0,75-84,T,NaN,NaN,N,1,N,Kujawsko-Pomorskie
5,2021-01-02,4,418,M,76.0,75-84,T,NaN,NaN,N,1,N,Kujawsko-Pomorskie
7,2021-01-02,4,464,M,67.0,65-74,T,NaN,NaN,N,1,N,Kujawsko-Pomorskie
...,...,...,...,...,...,...,...,...,...,...,...,...,...
58975,2021-12-13,32,3261,M,83.0,75-84,T,NaN,NaN,N,1,N,Zachodniopomorskie
58976,2021-12-13,32,3261,M,85.0,85-94,T,Pfizer,pelna_dawka,N,1,T,Zachodniopomorskie
58977,2021-12-13,32,3262,M,73.0,65-74,T,Pfizer,pelna_dawka,N,1,T,Zachodniopomorskie
58978,2021-12-13,32,3262,M,78.0,75-84,T,Pfizer,pelna_dawka,N,1,T,Zachodniopomorskie


In [102]:
from_date = datetime.date(2021,7,14)

dfM = prepare_df2(
    deaths_df = basiw.dfd.query('plec=="M"'), 
    x_col ='kat_wiek', 
    x_col_name = 'Grupa wiekowa',  
    denominator_col='mężczyzn', 
    from_date=from_date
    )

dfM = dfM[:-1].fillna(0)
dfM

,liczba_zaraportowanych_zgonow,mężczyzn,Zgony nie w pełni zaszczepionych mężczyzn,Zgony w pełni zaszczepionych mężczyzn,Grupa wiekowa,Zgony nie w pełni zaszczepionych mężczyzn na 100 tys. mężczyzn,Zgony w pełni zaszczepionych mężczyzn na 100 tys. mężczyzn
0-18,7,3546221.0,7,0.0,0-18,0.197393,0.000000
19-24,10,1221176.0,10,0.0,19-24,0.818883,0.000000
25-34,71,2740642.0,66,5.0,25-34,2.408195,0.182439
35-44,163,3092676.0,147,16.0,35-44,4.753165,0.517351
45-54,427,2393794.0,370,57.0,45-54,15.456635,2.381157
55-64,862,2435381.0,664,198.0,55-64,27.264728,8.130145
65-74,2032,1833485.0,1292,740.0,65-74,70.466898,40.360297
75-84,1898,693760.0,1176,722.0,75-84,169.511070,104.070572
85-94,1355,215975.0,878,477.0,85-94,406.528533,220.858896
95+,103,11563.0,80,23.0,95+,691.861974,198.910317


In [104]:
x_col = "Grupa wiekowa"
title_str = "w grupach wiekowych w Polsce," #"w województwach"
filename_str = "wiek_M"
denominator_col = "mężczyzn"


title = "Liczba zgonów " + denominator_col + " na COVID-19 " + title_str + " na 100 tys. "+ denominator_col +
        "<br>w podziale na w pełni zaszczepionych i nie w pełni zaszczepionych<br>(od " + str(from_date) + ")",

plot_df2(
    x_col=x_col, 
    denominator_col=denominator_col, 
    title_str=title_str , 
    filename_str=filename_str,  
    from_date = from_date, 
    df = dfM, 
    image_dir = basiw.image_dir
    )

In [105]:
from_date = datetime.date(2021,7,14)

dfK = prepare_df2(
    deaths_df = basiw.dfd.query('plec=="K"'), 
    x_col ='kat_wiek', 
    x_col_name = 'Grupa wiekowa',  
    denominator_col='kobiet', 
    from_date=from_date
    )

dfK = dfK[:-1].fillna(0)
dfK

,liczba_zaraportowanych_zgonow,kobiet,Zgony nie w pełni zaszczepionych kobiet,Zgony w pełni zaszczepionych kobiet,Grupa wiekowa,Zgony nie w pełni zaszczepionych kobiet na 100 tys. kobiet,Zgony w pełni zaszczepionych kobiet na 100 tys. kobiet
0-18,2,3362871.0,1,1.0,0-18,0.029736,0.029736
19-24,8,1169959.0,7,1.0,19-24,0.598312,0.085473
25-34,28,2637687.0,26,2.0,25-34,0.985712,0.075824
35-44,85,3008948.0,76,9.0,35-44,2.525800,0.299108
45-54,192,2395882.0,169,23.0,45-54,7.053770,0.959980
55-64,598,2668881.0,503,95.0,55-64,18.846850,3.559544
65-74,1526,2362488.0,1133,393.0,65-74,47.957916,16.635005
75-84,2022,1218241.0,1529,493.0,75-84,125.508828,40.468183
85-94,2176,540453.0,1739,437.0,85-94,321.767110,80.858095
95+,264,40781.0,219,45.0,95+,537.014786,110.345504


In [107]:
x_col = "Grupa wiekowa"
title_str = "w grupach wiekowych w Polsce," #"w województwach"
filename_str = "wiek_K"
denominator_col = "kobiet"
df_to_plot = dfK

plot_df2(
    x_col=x_col, 
    denominator_col=denominator_col, 
    title_str=title_str , 
    filename_str=filename_str,  
    from_date = from_date, 
    df = df_to_plot, 
    image_dir = basiw.image_dir
    )